In [1]:
%load_ext autoreload
%autoreload 2

In [6]:
import sys
import os
from icecream import ic
sys.path.append("../")
from rbsimclr_trainer import *

In [10]:
# Get dataset
dataset = ContrastiveLearningDataset("./datasets")
num_views = 2
print(DEVICE)
train_dataset = dataset.get_dataset('cifar10_tri', num_views)
val_dataset = dataset.get_dataset('cifar10_val', num_views)
# Batch size config
bsz_tri, bsz_val = 128, 128
train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=bsz_tri, shuffle=True,
    num_workers=2, pin_memory=True, drop_last=True)
val_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=bsz_val, shuffle=True,
    num_workers=2, pin_memory=True, drop_last=True)

# Model and training config
model = RBSimCLR(256).to(DEVICE)
optimizer = torch.optim.Adam(
    model.parameters(), lr=1e-3, betas=(0.5, 0.999))

# Warmup scheduler
warmupscheduler = torch.optim.lr_scheduler.LambdaLR(
    optimizer, lambda epoch: (epoch+1)/10.0, verbose=True)
# SCHEDULER FOR COSINE DECAY
mainscheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(
    optimizer, 500, eta_min=0.05, last_epoch=-1, verbose=True)
scheduler = {
    'warmupscheduler': warmupscheduler,
    'mainscheduler': mainscheduler
}

# LOSS FUNCTION
tri_criterion = RBSimCLRLoss(batch_size=bsz_tri, temperature=0.5)
val_criterion = RBSimCLRLoss(batch_size=bsz_val, temperature=0.5)
criterion = {
    'tri_criterion': tri_criterion,
    'val_criterion': val_criterion
}

# Logger & Writer
logger = Logger()
writer = SummaryWriter("RBSimCLR")

# Train
RBSimCLR_trainer(model, train_loader, val_loader,
                    optimizer, scheduler, criterion, writer, logger, bsz_tri, bsz_val)


cpu
Files already downloaded and verified
Files already downloaded and verified


ic| f"Checkpoint {current_epoch} saved!": 'Checkpoint 0 saved!'


Adjusting learning rate of group 0 to 1.0000e-04.
Epoch 00000: adjusting learning rate of group 0 to 1.0000e-03.
Device: cpu


FileNotFoundError: [Errno 2] No such file or directory: './checkpoint/RBSimCLR_epoch_0_checkpoint.pt'